In [ ]:
# Import the required libraries and dependencies
import pandas as pd
from pathlib import Path

## Import data 

In [ ]:
# Import indiegogo dataset
indiegogo_df = pd.read_csv(
    Path('./Resources/indiegogo_data/indiegogo.csv')
)

# Fix ID column
indiegogo_df.rename(columns={'project_id' : 'ID'}, inplace=True)

# Set the index as the ID
indiegogo_df.set_index('ID', inplace=True)

# View head
indiegogo_df.head(2)

## Rename cols

In [ ]:
# Rename columns to match the kickstarter df
indiegogo_df.rename(columns={'category' : 'main_category', 'title' : 'name', 'amount_raised_usd' : 'usd_pledged_real', 'goal_usd' : 'usd_goal_real'}, inplace=True)

## Clean df

In [ ]:
# fix dates
indiegogo_df['launched'] = pd.to_datetime(indiegogo_df['date_launch'] + ' ' + indiegogo_df['time_launch'])
indiegogo_df['deadline'] = pd.to_datetime(indiegogo_df['date_end'] + ' ' + indiegogo_df['time_end'])

## Create dfs from selected columns

In [ ]:
# List columns to drop
indiegogo_cols_drop = ['year_end',
'month_end',
'day_end',
'amount_raised',
'in_demand',
'year_launch',
'month_launch',
'day_launch',
'tagline',
'url',
'australia',
'canada',
'switzerland',
'denmark',
'western_europe',
'great_britain',
'hong_kong',
'norway',
'sweden',
'singapore',
'united_states',
'education',
'productivity',
'energy_greentech',
'wellness',
'comics',
'fashion_wearables',
'video_games',
'photography',
'tv_shows',
'dance_theater',
'phones_accessories',
'audio',
'film',
'transportation',
'art',
'environment',
'writing_publishing',
'music',
'travel_outdoors',
'health_fitness',
'tabletop_games',
'home',
'local_business',
'food_beverage',
'culture',
'human_rights',
'podcasts_vlogs',
'camera_gear',
'time_end',
'time_launch',
'date_launch',
'date_end',
'jan',
'feb',
'mar',
'apr',
'may',
'jun',
'jul',
'aug',
'sep',
'oct',
'nov',
'dec',
'tperiod']

# Selected columns indiegogo df
indiegogo_selected_cols_df = indiegogo_df.drop(indiegogo_cols_drop, axis=1)

## Add new cols

In [ ]:
# Duration
indiegogo_selected_cols_df['duration'] = indiegogo_selected_cols_df['deadline'] - indiegogo_selected_cols_df['launched'] 
indiegogo_selected_cols_df['duration'] = indiegogo_selected_cols_df['duration'].dt.days

# Daily Goal 
indiegogo_selected_cols_df['daily_goal'] = round(indiegogo_selected_cols_df['usd_goal_real'] / indiegogo_selected_cols_df['duration'],2)

# Daily Pledged
indiegogo_selected_cols_df['daily_pledged'] = round(indiegogo_selected_cols_df['usd_pledged_real'] / indiegogo_selected_cols_df['duration'],2)

# Funded Percentage
indiegogo_selected_cols_df['funded_percent'] = round(indiegogo_selected_cols_df['usd_pledged_real'] / indiegogo_selected_cols_df['usd_goal_real'],4)

# Fix types
indiegogo_selected_cols_df = indiegogo_selected_cols_df.astype({"funded_percent": 'float'})

## Reorder cols

In [ ]:
# Base column names for reordering
base_order = ['name', 'main_category', 'currency', 'usd_goal_real', 'usd_pledged_real', 'deadline', 'launched', 'state', 'funded_percent','duration', 'daily_goal', 'daily_pledged']
indiegogo_order = base_order

# Reorder each df
indiegogo_selected_cols_df = indiegogo_selected_cols_df[indiegogo_order]

## Write clean data to files

In [ ]:
indiegogo_selected_cols_df.to_csv('./Resources/indiegogo_data_clean/indiegogo-projects.csv')